In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
import csv
import requests
from io import StringIO

# URL of the input file
input_url = "https://raw.githubusercontent.com/Benz-Poobua/ESS_523/main/HW1_GNSS_data.txt"

# Output file (CSV)
output_file = "GNSS_data.csv"

# Fetching data from URL
response = requests.get(input_url)

# Checking if request was successful
if response.status_code == 200:
    # Read the data from the response
    data = response.text

    # Read the data and write it to CSV
    with open(output_file, 'w', newline='') as outfile:
        # Create a CSV writer object
        writer = csv.writer(outfile)
        
        # Write the header row
        writer.writerow(["x_position(km)", "Surface_velocity(mm/yr)", "uncertainty(mm/yr)"])
        
        # Skip the header lines
        data_lines = data.strip().split('\n')[3:]
        
        # Write the data rows
        for line in data_lines:
            # Split the line based on whitespace
            columns = line.split()
            # Write the columns to CSV
            writer.writerow(columns)

    print("Data converted to CSV successfully. Converted data saved to", output_file)
else:
    print("Failed to retrieve data from the URL.")


Data converted to CSV successfully. Converted data saved to converted_data.csv
